## Dram Shop Example

As discussed in the lecture and the reading, Association Analysis is a technique to find common sets of items that co-occur within a data set. 

--- 

### Data Format for mlxtend's Association Analysis

`mlxtend` requires the input data to be in a specific format for association rule mining:

1. Transactional Data: Each transaction is a list of items purchased together. For the Wedge, we can define "transactions" as a trip to the grocery store. For the Dram Shop, it will make more sense to think of a "transaction" as a customer's history, since many customers buy only one or two items on a visit. 
1. One-Hot Encoded DataFrame: A Pandas DataFrame where each row represents a transaction and each column represents an item. The cells contain binary values indicating the presence (True/1) or absence (False/0) of an item in a transaction.

For `mlxtend.frequent_patterns.apriori` and `association_rules` functions, the data must be in a one-hot encoded format.

#### Transactional Data

Each transaction in your dataset should be represented as a list of items purchased together.

#### One-Hot Encoded DataFrame

- **Rows**: Each row corresponds to a transaction.
- **Columns**: Each column represents an item in the dataset.
- **Values**: Binary indicators (`1` or `0`) showing whether an item is present in a transaction.

##### Example of One-Hot Encoded DataFrame

| Transaction_ID | Item_A | Item_B | Item_C | Item_D |
|----------------|--------|--------|--------|--------|
| 1              | 1      | 0      | 1      | 0      |
| 2              | 0      | 1      | 1      | 1      |
| 3              | 1      | 1      | 0      | 0      |



In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from google.cloud import bigquery


<frozen importlib._bootstrap>:488: DeprecationWarning: Type google.protobuf.pyext._message.ScalarMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.
<frozen importlib._bootstrap>:488: DeprecationWarning: Type google.protobuf.pyext._message.MessageMapContainer uses PyType_Spec with a metaclass that has custom tp_new. This is deprecated and will no longer be allowed in Python 3.14.


### Getting Data

Let's connect to GBQ and get data for the Dram Shop. The Dram Shop analysis is unusual since there are rarely many items in a "market basket", so I am considering a "transaction" to be a customer's history. Make sure to read my comments in the query below. 

Your query for the Wedge will be a bit simpler, because you can let actual transactions be the "market basket". 

In [2]:
project_id = "umt-msba"
client = bigquery.Client(project=project_id)

In [3]:
query = """
    SELECT
        COALESCE(o.customer_id, o.square_unique_id) AS customer_identifier,
        oi.catalog_object_id,
        id.clean_item_name,
        id.clean_category_name,
        COUNT(*) AS item_count,
        ROUND(SUM(oi.total_money)/100, 2) AS total_item_spend
    FROM `umt-msba.dram_shop.orders_*` o
    JOIN `umt-msba.dram_shop.order_items_*` oi ON o.order_id = oi.order_id
    JOIN `umt-msba.dram_shop.item_data` id ON oi.catalog_object_id = id.variant_id
    WHERE COALESCE(o.customer_id, o.square_unique_id) != "" 
      -- This next section limits our data to only the top 1000 items
      -- from the last 3 years. You'll want to do some similar "top item" 
      -- filtering in your Wedge analysis. 
      AND id.clean_item_name IN (
          SELECT clean_item_name
          FROM `umt-msba.dram_shop.vw_item_year_month`
          WHERE year IN (2022, 2023, 2024)
          GROUP BY clean_item_name
          ORDER BY SUM(total_sales) DESC
          LIMIT 1000
      )
      -- This line limits our data to only customers who have purchased
      -- multiple items. You probably *don't* need to do something like this
      -- on the Wedge, since the profile of shoppers is so different.
      AND COALESCE(o.customer_id, o.square_unique_id) IN (
          SELECT COALESCE(o.customer_id, o.square_unique_id)
          FROM `umt-msba.dram_shop.orders_*` o
          JOIN `umt-msba.dram_shop.order_items_*` oi ON o.order_id = oi.order_id
          GROUP BY COALESCE(o.customer_id, o.square_unique_id)
          HAVING COUNT(DISTINCT oi.catalog_object_id) > 1
      )
    GROUP BY customer_identifier, oi.catalog_object_id, id.clean_item_name, id.clean_category_name
"""


In [4]:
query_job = client.query(query)
results = query_job.result()

cust_item_sales = results.to_dataframe()

/opt/anaconda3/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


The data frame `cust_item_sales` holds the customer and item names. You'll do something similar for the Wedge with `card_no` and `description`.

In [5]:
cust_item_sales.sample(n=5)

,customer_identifier,catalog_object_id,clean_item_name,clean_category_name,item_count,total_item_spend
141920,GJXXWQVHB52G6PRFX3B8C9DXG8,7GWYPRMLEWI7VCU2MZP23FLN,Blackfoot Single Malt IPA,IPA - Draught,1,5.5
64751,AR3KK1XVZ95X13Y3SPEMPTSCZR,FR3WP7QIKOQ4CYBEFJSPZ7ZQ,Pigs Ass Porter,Porters/Stouts - Draught,1,4.5
262433,AWSFH9JZ8123KC78F5VFHTM0GM,UU5DKNHA7VUQWHU6HC6KI7KR,Do Tell Sparkling Ale,Ambers/Pales - Bottled,3,24.0
94025,BBFRYEN5ZN3KS4M6QWSFF0CAMG,JXIRQTOU6362YBCZMVKNCIBV,BA Farmhouse Ale,Belgian - Draught,1,5.0
125025,VJFH5WJPY97GZCZTTDB5QFBC0G,Z5ZBGQQXXZI7EDFP3TDNNMI7,Pink Slip,Lagers/Pils/Wheat - Draught,1,5.5


We can use these data to look at some interesting summary statistics about customer behavior.

In [6]:
cust_item_sales.groupby('clean_item_name')['item_count'].sum().sort_values(ascending=False).head(10)


clean_item_name
Blackfoot Single Malt IPA    35191
Super Pils                   10195
Pilsner                       6782
Grazing Clouds Hazy IPA       5933
IPA                           4510
Pear Cider                    4369
Helles Lager                  4167
Robot Panda Hazy IPA          2989
Grapefruit Radler             2735
All Day IPA                   2683
Name: item_count, dtype: Int64

In [7]:
cust_item_sales.groupby('clean_category_name')['total_item_spend'].sum().sort_values(ascending=False).head(20)

clean_category_name
IPA - Draught                  867541.95
Lagers/Pils/Wheat - Draught    440917.08
Wine - Draught                 220381.89
Cider - Draught                219019.02
Ambers/Pales - Draught         166928.66
Porters/Stouts - Draught       128460.82
Sour - Draught                 110778.78
Growlers                        45671.44
Belgian - Draught               34429.91
Hard Seltzer - Draught          20378.73
Red Wine - Bottled              19020.09
Seltzer                         18057.03
IPA - Bottled                   15538.00
Sparkling Wine - Bottled        15096.65
Seasonal/Event                  15072.66
Lagers/Pils/Wheat - Bottled     13606.38
White Wine - Bottled            13540.30
Rosé Wine - Bottled             11408.15
Swag                             8462.85
Soda - Draught                   6010.03
Name: total_item_spend, dtype: float64

Now we convert this data frame to our encoded "one hot" matrix.

In [8]:
transactions = cust_item_sales.groupby('customer_identifier')['clean_item_name'].apply(list).tolist()

# One-hot encoding
te = TransactionEncoder()
te_data = te.fit(transactions).transform(transactions)
ci_sales_encoded = pd.DataFrame(te_data, columns=te.columns_)



In [9]:

ci_sales_encoded.head()

,$5,$7,100 Degrees Shandy Inspired Farmhouse Ale,1000 Petals Pilsner,10G Pumpkin Spice Latte,14C Who's Whoo,1664,"20"" Brown","20,000 Leguas' Amber Wine Chardonnay",2424 Hazy Double,...,ZC Up Stream,ZM Breezy Does It,ZP 56 Counties,ZP Spruce Tip IPA,Zinfandel,Zip Hoody,Zymopunk Pilsner,|Z Populis Sauvignon Blanc - 2022| Sauvignon Blanc | Populis | 2022 |,¡Viva La Pineapple!,‘River Sand’ Fiano
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


You can play with `min_support` to get more or fewer rules. 

In [10]:
frequent_itemsets = apriori(ci_sales_encoded, min_support=0.003, use_colnames=True)

# Sort by support
frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)
print(frequent_itemsets.head())


      support                     itemsets
33   0.221901  (Blackfoot Single Malt IPA)
335  0.075053                 (Super Pils)
257  0.065808                    (Pilsner)
128  0.056108    (Grazing Clouds Hazy IPA)
250  0.049083                 (Pear Cider)


In [11]:
frequent_itemsets

,support,itemsets
33,0.221901,(Blackfoot Single Malt IPA)
335,0.075053,(Super Pils)
257,0.065808,(Pilsner)
128,0.056108,(Grazing Clouds Hazy IPA)
250,0.049083,(Pear Cider)
...,...,...
809,0.003003,"(Pilsner, Helles Lager, Grazing Clouds Hazy IPA)"
749,0.003003,"(Blackfoot Single Malt IPA, Robot Panda Hazy I..."
72,0.003003,(Cosmic Crisp)
150,0.003003,(Hell Lager)


In [12]:
rules = association_rules(frequent_itemsets, num_itemsets=len(frequent_itemsets), metric="confidence", min_threshold=0.5)


In [13]:

# Sort by lift
rules = rules.sort_values(by='lift', ascending=False)


In [14]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
104,"(Carlsberg Lager, Pilsner)",(Helles Lager),0.006078,0.045170,0.003094,0.508982,11.268149,1.0,0.002819,1.944593,0.916827,0.064248,0.485754,0.288738
94,"(Robot Panda Hazy IPA, Head Full of Dynamite)",(Grazing Clouds Hazy IPA),0.005405,0.056108,0.003239,0.599327,10.681738,1.0,0.002936,2.355765,0.911308,0.055590,0.575509,0.328531
77,"(IPA, Head Full of Dynamite)",(Grazing Clouds Hazy IPA),0.005951,0.056108,0.003476,0.584098,10.410318,1.0,0.003142,2.269506,0.909353,0.059335,0.559375,0.323025
110,"(Helles Lager, Kolsch)",(Pilsner),0.004513,0.065808,0.003076,0.681452,10.355200,1.0,0.002779,2.932654,0.907526,0.045737,0.659012,0.364094
116,"(IPA, Bodhizafa IPA)",(Grazing Clouds Hazy IPA),0.005278,0.056108,0.003057,0.579310,10.324990,1.0,0.002761,2.243679,0.907939,0.052418,0.554303,0.316901


In [17]:
def print_association_rules(rules_df, min_support=0.3, min_confidence=0.7):
    # Filter rules based on the minimum support and confidence thresholds
    filtered_rules = rules_df[
        (rules_df['support'] >= min_support) & (rules_df['confidence'] >= min_confidence)
    ]
    
    # Print the frequent itemsets with their support
    print("Itemsets with Support >= {:.2f}:".format(min_support))
    for _, row in filtered_rules.iterrows():
        antecedents = ", ".join(row['antecedents'])
        consequents = ", ".join(row['consequents'])
        print(f"Item: ({antecedents}), Support: {row['support']:.3f}")

    print("\n------------------------ RULES: ------------------------\n")
    
    # Print the association rules with confidence and lift
    for _, row in filtered_rules.iterrows():
        antecedents = ", ".join(row['antecedents'])
        consequents = ", ".join(row['consequents'])
        print(
            f"Rule: ({antecedents}) ==> ({consequents}), Confidence: {row['confidence']:.3f}, Lift: {row['lift']:.3f}"
        )


In [18]:
print_association_rules(rules, min_support=0.005, min_confidence=0.3)


Itemsets with Support >= 0.01:
Item: (Pilsner, Helles Lager), Support: 0.007
Item: (Super Pils, Helles Lager), Support: 0.007
Item: (IPA, Super Pils), Support: 0.006
Item: (IPA, Pilsner), Support: 0.005
Item: (IPA, Grazing Clouds Hazy IPA), Support: 0.006
Item: (Grazing Clouds Hazy IPA, Head Full of Dynamite), Support: 0.005
Item: (Robot Panda Hazy IPA, Grazing Clouds Hazy IPA), Support: 0.006
Item: (Super Pils, Grazing Clouds Hazy IPA), Support: 0.007
Item: (Pilsner, Grazing Clouds Hazy IPA), Support: 0.005
Item: (Lagunitas IPA), Support: 0.006
Item: (Chair 4 IPA), Support: 0.008
Item: (Pilsner, Helles Lager), Support: 0.006
Item: (60 Minute IPA), Support: 0.006
Item: (Yaak Attack IPA), Support: 0.007
Item: (Lush IPA), Support: 0.008
Item: (Wanderlust IPA), Support: 0.007
Item: (Lucille IPA), Support: 0.006
Item: (Celebration), Support: 0.011
Item: (Breakside IPA), Support: 0.005
Item: (Stone IPA), Support: 0.010

------------------------ RULES: ------------------------

Rule: (Pilsne